# Masque Causal et Attention Autoregressive

## Introduction

Bienvenue dans ce notebook sur le **masque causal**, le mécanisme qui distingue GPT de BERT!

### Objectifs pédagogiques

Dans ce notebook, vous allez:
1. Comprendre le concept de masque causal (triangulaire)
2. Implémenter le masque from scratch avec NumPy
3. Implémenter le masque avec PyTorch (`torch.tril()`)
4. Visualiser l'effet du masque sur l'attention
5. Comparer BERT (bidirectionnel) et GPT (causal)
6. Comprendre la génération autoregressive

### Qu'est-ce qu'un masque causal?

Un **masque causal** est une matrice triangulaire qui empêche chaque token de voir les tokens futurs. C'est le mécanisme fondamental des modèles génératifs comme GPT.

**Analogie:** Imaginez que vous lisez un livre mot par mot. À chaque instant, vous ne pouvez voir que les mots que vous avez déjà lus, pas ceux qui viennent après. C'est exactement ce que fait le masque causal!

**Pourquoi est-ce important?**
- **GPT (génération):** Doit prédire le prochain mot sans voir le futur → masque causal
- **BERT (compréhension):** Peut voir toute la phrase pour comprendre → pas de masque

## 1. Formule Mathématique du Masque Causal

### Définition du Masque

Le masque causal est une matrice $M \in \{0, 1\}^{n \times n}$ définie par:

$M_{ij} = \begin{cases} 1 & \text{si } j \leq i \\ 0 & \text{si } j > i \end{cases}$

**Interprétation:**
- $M_{ij} = 1$ signifie: "le token $i$ peut voir le token $j$"
- $M_{ij} = 0$ signifie: "le token $i$ ne peut PAS voir le token $j$"
- Condition $j \leq i$: on ne peut voir que le passé et le présent

### Exemple pour n=5

$M = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \\ 1 & 1 & 0 & 0 & 0 \\ 1 & 1 & 1 & 0 & 0 \\ 1 & 1 & 1 & 1 & 0 \\ 1 & 1 & 1 & 1 & 1 \end{bmatrix}$

C'est une **matrice triangulaire inférieure** (lower triangular matrix).

### Application dans l'Attention

Le masque est appliqué aux scores d'attention AVANT le softmax:

$\text{scores}_{ij} = \begin{cases} \frac{Q_i K_j^T}{\sqrt{d_k}} & \text{si } M_{ij} = 1 \\ -\infty & \text{si } M_{ij} = 0 \end{cases}$

**Pourquoi $-\infty$?**

$\text{softmax}(-\infty) = \frac{e^{-\infty}}{\sum e^{\cdot}} = \frac{0}{\sum e^{\cdot}} = 0$

Donc les positions masquées auront une attention nulle!

### Formule Complète avec Masque

$\text{Attention}(Q, K, V, M) = \text{softmax}\left(\text{mask}\left(\frac{QK^T}{\sqrt{d_k}}, M\right)\right)V$

Où $\text{mask}(S, M)$ remplace les positions $M_{ij}=0$ par $-\infty$.

In [ ]:
# Imports
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from typing import Optional, Tuple

# Ajouter le chemin vers src/
sys.path.append('../..')

# Importer nos modules
from src.attention.masking import (
    create_causal_mask_from_scratch,
    create_causal_mask_manual,
    create_causal_mask,
    visualize_causal_mask,
    compare_attention_patterns,
    visualize_mask_effect_on_attention
)
from src.attention.scaled_dot_product import ScaledDotProductAttention

# Configuration pour les visualisations
plt.style.use('default')
sns.set_palette("husl")

# Seed pour la reproductibilité
np.random.seed(42)
torch.manual_seed(42)

print("✓ Imports réussis!")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ Device disponible: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## 2. Implémentation From Scratch (NumPy)

Commençons par créer un masque causal avec NumPy pour comprendre la structure.

### 2.1 Méthode 1: Avec np.tril() (Recommandé)

In [ ]:
def create_causal_mask_numpy(seq_len: int) -> np.ndarray:
    """
    Crée un masque causal avec np.tril() (triangular lower).
    
    np.tril() retourne la partie triangulaire inférieure d'une matrice.
    C'est la méthode la plus simple et efficace.
    
    Args:
        seq_len: Longueur de la séquence
    
    Returns:
        Masque causal (seq_len, seq_len)
        1 = autoriser l'attention, 0 = bloquer l'attention
    """
    # np.tril() garde seulement la partie triangulaire inférieure
    mask = np.tril(np.ones((seq_len, seq_len)))
    return mask

# Test avec seq_len = 6
seq_len = 6
mask = create_causal_mask_numpy(seq_len)

print("=" * 60)
print("MASQUE CAUSAL (NumPy)")
print("=" * 60)
print(f"\nShape: {mask.shape}")
print(f"\nMasque (1=autorisé, 0=bloqué):\n")
print(mask)

print("\n" + "-" * 60)
print("INTERPRÉTATION")
print("-" * 60)
for i in range(seq_len):
    visible_positions = np.where(mask[i] == 1)[0]
    print(f"Token {i} peut voir les positions: {list(visible_positions)} "
          f"(total: {len(visible_positions)} tokens)")

### 2.2 Méthode 2: Construction Manuelle (Pédagogique)

Pour bien comprendre, construisons le masque manuellement avec des boucles.

In [ ]:
def create_causal_mask_manual_demo(seq_len: int) -> np.ndarray:
    """
    Crée un masque causal manuellement (version pédagogique).
    
    Cette version montre explicitement la logique avec des boucles.
    """
    # Initialiser une matrice de zéros
    mask = np.zeros((seq_len, seq_len))
    
    # Remplir la partie triangulaire inférieure
    for i in range(seq_len):
        for j in range(seq_len):
            if j <= i:  # Position j est dans le passé ou présent
                mask[i, j] = 1.0
            # else: mask[i, j] reste 0 (futur bloqué)
    
    return mask

# Comparer les deux méthodes
mask_tril = create_causal_mask_numpy(5)
mask_manual = create_causal_mask_manual_demo(5)

print("Méthode 1 (np.tril):")
print(mask_tril)
print("\nMéthode 2 (manuelle):")
print(mask_manual)
print(f"\nLes deux méthodes donnent le même résultat? {np.allclose(mask_tril, mask_manual)}")

## 3. Implémentation PyTorch

Maintenant, créons le masque avec PyTorch en utilisant `torch.tril()`.

### 3.1 Fonction avec torch.tril()

In [ ]:
def create_causal_mask_pytorch(seq_len: int, device: torch.device = torch.device('cpu')) -> torch.Tensor:
    """
    Crée un masque causal avec PyTorch.
    
    torch.tril() - Triangular Lower:
        - Retourne la partie triangulaire inférieure d'une matrice
        - Tous les éléments au-dessus de la diagonale sont mis à 0
        - Optimisé GPU et différentiable
    
    Args:
        seq_len: Longueur de la séquence
        device: Device PyTorch (cpu ou cuda)
    
    Returns:
        Masque causal (seq_len, seq_len)
    """
    # torch.tril() retourne la partie triangulaire inférieure
    mask = torch.tril(torch.ones(seq_len, seq_len, device=device))
    return mask

# Test sur CPU
print("=" * 60)
print("MASQUE CAUSAL (PyTorch)")
print("=" * 60)

mask_torch = create_causal_mask_pytorch(6)
print(f"\nShape: {mask_torch.shape}")
print(f"Device: {mask_torch.device}")
print(f"Dtype: {mask_torch.dtype}")
print(f"\nMasque:\n{mask_torch}")

# Test sur GPU si disponible
if torch.cuda.is_available():
    print("\n" + "-" * 60)
    print("Test sur GPU")
    print("-" * 60)
    mask_gpu = create_causal_mask_pytorch(6, device=torch.device('cuda'))
    print(f"Device: {mask_gpu.device}")
    print(f"CPU vs GPU identiques? {torch.allclose(mask_torch, mask_gpu.cpu())}")
else:
    print("\nGPU non disponible, test sur CPU uniquement")

## 4. Visualisation du Masque Causal

Visualisons le masque sous forme de heatmap pour mieux comprendre sa structure.

In [ ]:
# Visualiser le masque causal
visualize_causal_mask(seq_len=8)

### Interprétation de la Visualisation

**Observations:**
- La partie **triangulaire inférieure** (vert) = attention autorisée
- La partie **triangulaire supérieure** (rouge) = attention bloquée
- La **diagonale** = chaque token peut se voir lui-même

**Lecture:**
- **Ligne i** = ce que le token i peut voir
- **Colonne j** = qui peut voir le token j

**Exemple:**
- Token 0: ne voit que lui-même (1 token visible)
- Token 3: voit tokens 0, 1, 2, 3 (4 tokens visibles)
- Token 7: voit tous les tokens 0-7 (8 tokens visibles)

## 5. Effet du Masque sur l'Attention

Voyons comment le masque transforme les scores d'attention.

### 5.1 Attention Sans Masque vs Avec Masque

In [ ]:
# Créer des données de test
torch.manual_seed(42)
batch_size = 1
seq_len_test = 5
d_k = 8

Q = torch.randn(batch_size, seq_len_test, d_k)
K = torch.randn(batch_size, seq_len_test, d_k)
V = torch.randn(batch_size, seq_len_test, d_k)

# Créer le module d'attention
attention = ScaledDotProductAttention(d_k=d_k)

# Créer le masque causal
causal_mask = create_causal_mask_pytorch(seq_len_test)

print("=" * 60)
print("COMPARAISON: SANS MASQUE vs AVEC MASQUE")
print("=" * 60)

# Attention SANS masque
print("\n1. Attention SANS masque (bidirectionnelle - BERT)")
print("-" * 60)
output_no_mask, weights_no_mask = attention(Q, K, V, mask=None)
print("Poids d'attention (tous les tokens se voient):")
print(weights_no_mask[0])

# Attention AVEC masque causal
print("\n2. Attention AVEC masque causal (autorégressif - GPT)")
print("-" * 60)
output_masked, weights_masked = attention(Q, K, V, mask=causal_mask)
print("Poids d'attention (masque causal appliqué):")
print(weights_masked[0])

print("\n" + "=" * 60)
print("OBSERVATIONS")
print("=" * 60)
print("✓ Sans masque: Tous les poids sont non-nuls")
print("✓ Avec masque: La partie supérieure droite est nulle (futur bloqué)")
print("✓ Chaque token ne voit que lui-même et les tokens précédents")

### 5.2 Statistiques Détaillées

In [ ]:
# Analyser les différences
print("\n" + "=" * 60)
print("STATISTIQUES PAR POSITION")
print("=" * 60)
print(f"\n{'Position':<10} {'Sans masque':<15} {'Avec masque':<15} {'Différence'}")
print("-" * 60)

for i in range(seq_len_test):
    visible_no_mask = (weights_no_mask[0, i] > 0).sum().item()
    visible_masked = (weights_masked[0, i] > 0).sum().item()
    diff = visible_no_mask - visible_masked
    print(f"Token {i:<5} {visible_no_mask:<15} {visible_masked:<15} {diff}")

print("\n" + "-" * 60)
print("INTERPRÉTATION")
print("-" * 60)
print("• Sans masque: Chaque token voit tous les tokens (5 tokens)")
print("• Avec masque: Token i voit (i+1) tokens (positions 0 à i)")
print("• Différence: Nombre de tokens futurs bloqués")

## 6. Visualisation Complète de l'Effet du Masque

Visualisons les 4 étapes: scores bruts → masque → attention sans masque → attention avec masque.

In [ ]:
# Utiliser la fonction de visualisation complète
visualize_mask_effect_on_attention(seq_len=5)

## 7. Comparaison BERT vs GPT

Comparons côte à côte les deux architectures pour comprendre leur différence fondamentale.

### 7.1 Patterns d'Attention

In [ ]:
# Visualiser la comparaison BERT vs GPT
compare_attention_patterns(seq_len=6)

### 7.2 Différences Clés

| Aspect | BERT | GPT |
|--------|------|-----|
| **Type d'attention** | Bidirectionnelle | Causale (autoregressive) |
| **Masque** | Aucun | Triangulaire inférieur |
| **Vision** | Tous les tokens | Seulement passé + présent |
| **Tâche principale** | Compréhension | Génération |
| **Exemple d'usage** | Classification, Q&A | Génération de texte |
| **Architecture** | Encodeur | Décodeur |

### 7.3 Code Comparatif

In [ ]:
print("=" * 60)
print("CODE COMPARATIF: BERT vs GPT")
print("=" * 60)

print("\n# BERT: Attention Bidirectionnelle")
print("-" * 60)
print("""\n# Pas de masque - tous les tokens se voient
output_bert = attention(Q, K, V, mask=None)

# Utilisation: Comprendre le contexte complet
# Exemple: "Le [MASK] mange la souris" → prédire "chat"
# Le modèle voit "mange" et "souris" pour deviner "chat"
""")

print("\n# GPT: Attention Causale")
print("-" * 60)
print("""\n# Masque causal - empêche de voir le futur
causal_mask = torch.tril(torch.ones(seq_len, seq_len))
output_gpt = attention(Q, K, V, mask=causal_mask)

# Utilisation: Générer le prochain token
# Exemple: "Le chat mange" → prédire "la"
# Le modèle ne voit que "Le chat mange", pas la suite
""")

print("\n" + "=" * 60)
print("RÉSUMÉ")
print("=" * 60)
print("\n✓ BERT = Encodeur = Bidirectionnel = Compréhension")
print("✓ GPT = Décodeur = Causal = Génération")
print("✓ La seule différence: présence/absence du masque causal!")

## 8. Génération Autoregressive

Comprenons comment le masque causal permet la génération autoregressive.

### 8.1 Concept de Génération Autoregressive

**Définition:** Générer un token à la fois, en utilisant les tokens précédents comme contexte.

**Processus:**
1. Commencer avec un prompt: "Le chat"
2. Prédire le prochain token: "mange"
3. Ajouter au contexte: "Le chat mange"
4. Prédire le suivant: "la"
5. Continuer jusqu'à un token de fin

**Rôle du masque causal:**
- À l'étape 2, le modèle ne doit voir que "Le chat"
- À l'étape 4, le modèle ne doit voir que "Le chat mange"
- Le masque garantit qu'on ne triche pas en voyant le futur!

### 8.2 Simulation de Génération

In [ ]:
def simulate_autoregressive_generation():
    """
    Simule la génération autoregressive pour comprendre le rôle du masque.
    """
    # Tokens de la phrase complète (pour simulation)
    tokens = ["Le", "chat", "mange", "la", "souris"]
    
    print("=" * 60)
    print("SIMULATION: GÉNÉRATION AUTOREGRESSIVE")
    print("=" * 60)
    
    # Simuler la génération étape par étape
    for step in range(1, len(tokens)):
        context = tokens[:step]
        next_token = tokens[step]
        
        print(f"\nÉtape {step}:")
        print("-" * 60)
        print(f"  Contexte visible: {' '.join(context)}")
        print(f"  Longueur du contexte: {len(context)} tokens")
        print(f"  Token à prédire: '{next_token}'")
        print(f"  Masque: Token {step-1} peut voir positions 0 à {step-1}")
        
        # Visualiser le masque pour cette étape
        mask_step = np.tril(np.ones((step, step)))
        print(f"  Masque d'attention ({step}×{step}):")
        print(f"  {mask_step[-1]}  ← Dernière ligne (token actuel)")
    
    print("\n" + "=" * 60)
    print("OBSERVATIONS")
    print("=" * 60)
    print("✓ À chaque étape, on ne voit que le passé")
    print("✓ Le masque grandit avec la séquence")
    print("✓ C'est comme lire un livre mot par mot!")

simulate_autoregressive_generation()

### 8.3 Exemple Concret avec Attention

In [ ]:
# Créer une séquence d'exemple
torch.manual_seed(42)
seq_len_gen = 4
d_k_gen = 8

# Simuler des embeddings pour "Le chat mange la"
Q_gen = torch.randn(1, seq_len_gen, d_k_gen)
K_gen = torch.randn(1, seq_len_gen, d_k_gen)
V_gen = torch.randn(1, seq_len_gen, d_k_gen)

attention_gen = ScaledDotProductAttention(d_k=d_k_gen)
mask_gen = torch.tril(torch.ones(seq_len_gen, seq_len_gen))

_, weights_gen = attention_gen(Q_gen, K_gen, V_gen, mask=mask_gen)

# Visualiser
tokens_gen = ["Le", "chat", "mange", "la"]

plt.figure(figsize=(8, 6))
sns.heatmap(
    weights_gen[0].detach().numpy(),
    xticklabels=tokens_gen,
    yticklabels=tokens_gen,
    cmap='YlOrRd',
    annot=True,
    fmt='.3f',
    cbar_kws={'label': 'Poids d\'attention'},
    vmin=0,
    vmax=1
)
plt.xlabel('Keys (ce qu\'on regarde)', fontsize=12)
plt.ylabel('Queries (qui regarde)', fontsize=12)
plt.title('Attention Autoregressive: "Le chat mange la"', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nInterprétation:")
print("-" * 60)
print("• 'Le' (ligne 0): ne voit que lui-même")
print("• 'chat' (ligne 1): voit 'Le' et 'chat'")
print("• 'mange' (ligne 2): voit 'Le', 'chat', 'mange'")
print("• 'la' (ligne 3): voit tous les tokens précédents")
print("\n✓ Chaque token utilise tout le contexte disponible (passé)")
print("✓ Mais ne triche jamais en voyant le futur!")

## 9. Exercices Pratiques

### Exercice 1: Créer un Masque Personnalisé

Créez un masque qui permet à chaque token de voir:
- Lui-même
- Le token précédent
- Mais pas les autres tokens

**Exemple pour seq_len=5:**
```
[[1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 1, 1, 0, 0],
 [0, 0, 1, 1, 0],
 [0, 0, 0, 1, 1]]
```

In [ ]:
# TODO: Exercice 1 - Masque personnalisé
def create_local_mask(seq_len: int) -> np.ndarray:
    """
    Crée un masque où chaque token voit seulement lui-même et le précédent.
    """
    # Votre code ici
    mask = np.zeros((seq_len, seq_len))
    # TODO: Remplir le masque
    
    return mask

# Test
local_mask = create_local_mask(5)
print("Masque local:")
print(local_mask)

# Visualiser
plt.figure(figsize=(6, 5))
sns.heatmap(local_mask, annot=True, fmt='.0f', cmap='RdYlGn', cbar=False, square=True)
plt.title('Masque Local (fenêtre de 2 tokens)')
plt.show()

### Exercice 2: Analyser l'Impact du Masque

Comparez les poids d'attention moyens avec et sans masque.

In [ ]:
# TODO: Exercice 2 - Impact du masque

# Créer des données aléatoires
torch.manual_seed(42)
Q_ex = torch.randn(10, 8, 16)  # 10 exemples, seq_len=8, d_k=16
K_ex = torch.randn(10, 8, 16)
V_ex = torch.randn(10, 8, 16)

attention_ex = ScaledDotProductAttention(d_k=16)
mask_ex = torch.tril(torch.ones(8, 8))

# Calculer l'attention avec et sans masque
_, weights_no_mask_ex = attention_ex(Q_ex, K_ex, V_ex, mask=None)
_, weights_masked_ex = attention_ex(Q_ex, K_ex, V_ex, mask=mask_ex)

# TODO: Calculer les statistiques
# - Poids moyen par position
# - Variance des poids
# - Entropie de la distribution

print("Statistiques à calculer:")
print("1. Poids moyen par position")
print("2. Variance des poids d'attention")
print("3. Nombre de poids non-nuls par position")

### Exercice 3: Masque pour Padding

Créez un masque qui combine:
1. Masque causal (pas de futur)
2. Masque de padding (ignorer les tokens de remplissage)

**Exemple:** Séquence de longueur 5, mais seulement 3 tokens réels (les 2 derniers sont du padding).

In [ ]:
# TODO: Exercice 3 - Masque combiné
def create_combined_mask(seq_len: int, real_len: int) -> torch.Tensor:
    """
    Crée un masque combinant causal et padding.
    
    Args:
        seq_len: Longueur totale de la séquence
        real_len: Nombre de tokens réels (le reste est du padding)
    
    Returns:
        Masque combiné
    """
    # TODO: Votre code ici
    # Indice: Combiner masque causal ET masque de padding
    
    pass

# Test
combined_mask = create_combined_mask(seq_len=5, real_len=3)
print("Masque combiné (causal + padding):")
print(combined_mask)
print("\nInterprétation:")
print("• Tokens 0-2: tokens réels (masque causal appliqué)")
print("• Tokens 3-4: padding (complètement masqués)")

## 10. Résumé

### Ce que nous avons appris

1. **Masque causal:**
   - Matrice triangulaire inférieure
   - Empêche de voir le futur
   - Formule: $M_{ij} = 1$ si $j \leq i$, sinon $0$

2. **Implémentation:**
   - NumPy: `np.tril(np.ones((n, n)))`
   - PyTorch: `torch.tril(torch.ones(n, n))`

3. **Application:**
   - Remplacer positions masquées par $-\infty$
   - Softmax transforme $-\infty$ en $0$
   - Résultat: attention nulle aux positions futures

4. **BERT vs GPT:**
   - **BERT:** Pas de masque → bidirectionnel → compréhension
   - **GPT:** Masque causal → autorégressif → génération

5. **Génération autoregressive:**
   - Générer un token à la fois
   - Utiliser les tokens précédents comme contexte
   - Le masque garantit qu'on ne triche pas!

### Formules Clés

**Masque causal:**
$M_{ij} = \begin{cases} 1 & \text{si } j \leq i \\ 0 & \text{si } j > i \end{cases}$

**Application dans l'attention:**
$\text{Attention}(Q, K, V, M) = \text{softmax}\left(\text{mask}\left(\frac{QK^T}{\sqrt{d_k}}, M\right)\right)V$

### Prochaines étapes

Dans le prochain notebook, nous verrons:
- **Multi-Head Attention:** Plusieurs têtes d'attention en parallèle
- Comment combiner plusieurs perspectives
- L'architecture complète du transformer

### Points clés à retenir

✓ Le masque causal est une matrice triangulaire inférieure
✓ Il empêche chaque token de voir le futur
✓ C'est la différence clé entre BERT (encodeur) et GPT (décodeur)
✓ Il permet la génération autoregressive
✓ Implémentation simple: `torch.tril()` ou `np.tril()`